# AQI Data Analysis and Model Training

This notebook covers:
1. Data Loading and Preprocessing
2. Exploratory Data Analysis (EDA)
3. Feature Engineering
4. Model Training and Comparison
5. Model Evaluation and Selection

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Libraries loaded successfully!")

## 1. Data Generation and Loading

Since we may not have real data, we'll generate synthetic AQI data for demonstration.

In [ ]:
# Generate synthetic AQI data
import random
from datetime import datetime, timedelta

def calculate_aqi_category(aqi):
    """Determine AQI category based on EPA standards."""
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Very Unhealthy"
    else:
        return "Hazardous"

def generate_data(n_samples=5000):
    """Generate synthetic AQI data."""
    data = []
    cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]
    
    for i in range(n_samples):
        # Generate realistic pollutant values
        pm25 = max(0, np.random.lognormal(mean=2.5, sigma=0.8))
        pm10 = pm25 * (1.2 + random.uniform(0, 0.5))
        co = max(0, np.random.exponential(scale=2))
        no2 = max(0, np.random.normal(25, 15))
        so2 = max(0, np.random.exponential(scale=5))
        o3 = max(0, np.random.normal(40, 20))
        
        # Calculate AQI (simplified: based on PM2.5)
        aqi = min(500, max(0, int(pm25 * 2)))
        
        data.append({
            'city': random.choice(cities),
            'pm25': round(pm25, 2),
            'pm10': round(pm10, 2),
            'co': round(co, 2),
            'no2': round(no2, 2),
            'so2': round(so2, 2),
            'o3': round(o3, 2),
            'aqi': aqi,
            'aqi_category': calculate_aqi_category(aqi),
            'timestamp': datetime.now() - timedelta(hours=i)
        })
    
    return pd.DataFrame(data)

# Generate data
df = generate_data(5000)
print(f"Generated {len(df)} samples")
df.head()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print("Dataset Shape:", df.shape)
print("\nColumn Types:")
print(df.dtypes)
print("\nBasic Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

In [ ]:
# AQI Category Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
category_counts = df['aqi_category'].value_counts()
colors = ['#00E400', '#FFFF00', '#FF7E00', '#FF0000', '#8F3F97', '#7E0023']
categories_order = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 
                   'Unhealthy', 'Very Unhealthy', 'Hazardous']

ax1 = axes[0]
category_counts.reindex(categories_order).plot(kind='bar', ax=ax1, color=colors[:len(category_counts)])
ax1.set_title('AQI Category Distribution', fontsize=14)
ax1.set_xlabel('Category')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=45)

# Pie chart
ax2 = axes[1]
category_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=colors[:len(category_counts)])
ax2.set_title('AQI Category Proportion', fontsize=14)
ax2.set_ylabel('')

plt.tight_layout()
plt.savefig('../data/aqi_category_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Pollutant Distributions
pollutants = ['pm25', 'pm10', 'co', 'no2', 'so2', 'o3']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, pollutant in enumerate(pollutants):
    axes[i].hist(df[pollutant], bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'{pollutant.upper()} Distribution', fontsize=12)
    axes[i].set_xlabel(pollutant.upper())
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('../data/pollutant_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df[pollutants + ['aqi']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn_r', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Between Pollutants and AQI', fontsize=14)
plt.tight_layout()
plt.savefig('../data/correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Box plots by AQI category
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, pollutant in enumerate(pollutants):
    df.boxplot(column=pollutant, by='aqi_category', ax=axes[i], rot=45)
    axes[i].set_title(f'{pollutant.upper()} by AQI Category', fontsize=12)
    axes[i].set_xlabel('')

plt.suptitle('')
plt.tight_layout()
plt.savefig('../data/pollutants_by_category.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Data Preprocessing

In [ ]:
# Prepare features and target
X = df[pollutants].copy()
y = df['aqi_category'].copy()

# Handle missing values (if any)
X = X.fillna(X.median())

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Classes:", le.classes_)
print("\nFeature shape:", X.shape)
print("Target distribution:")
print(pd.Series(y_encoded).value_counts())

In [ ]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

## 4. Model Training

In [ ]:
# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss')
}

# Train and evaluate all models
results = {}

for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Training {name}...")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_scaled, y_encoded, cv=5, scoring='accuracy')
    
    results[name] = {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1_score': f1,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'model': model
    }
    
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

## 5. Model Comparison

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    name: {
        'Accuracy': res['accuracy'],
        'Precision': res['precision'],
        'Recall': res['recall'],
        'F1 Score': res['f1_score'],
        'CV Mean': res['cv_mean'],
        'CV Std': res['cv_std']
    }
    for name, res in results.items()
}).T

comparison_df = comparison_df.sort_values('F1 Score', ascending=False)
print("\nModel Comparison (Sorted by F1 Score):")
print(comparison_df.round(4))

In [ ]:
# Visualization of model comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df))
width = 0.2

for i, metric in enumerate(metrics):
    bars = ax.bar(x + i*width, comparison_df[metric], width, label=metric)

ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(comparison_df.index, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1.1)

plt.tight_layout()
plt.savefig('../data/model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, (name, res) in enumerate(results.items()):
    y_pred = res['model'].predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=le.classes_, yticklabels=le.classes_)
    axes[idx].set_title(f'{name}\nConfusion Matrix')
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')

# Hide empty subplot
axes[-1].axis('off')

plt.tight_layout()
plt.savefig('../data/confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Best Model Selection and Saving

In [ ]:
# Select best model
best_model_name = comparison_df.index[0]  # First row has highest F1
best_model = results[best_model_name]['model']

print(f"\nBest Model: {best_model_name}")
print(f"F1 Score: {results[best_model_name]['f1_score']:.4f}")

# Detailed classification report
y_pred_best = best_model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best, target_names=le.classes_))

In [ ]:
# Save models
import os

models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)

# Save scaler
joblib.dump(scaler, f'{models_dir}/scaler.joblib')

# Save label encoder
joblib.dump(le, f'{models_dir}/label_encoder.joblib')

# Save all models
for name, res in results.items():
    safe_name = name.lower().replace(' ', '_')
    joblib.dump(res['model'], f'{models_dir}/{safe_name}.joblib')

# Save best model separately
joblib.dump(best_model, f'{models_dir}/best_model.joblib')

# Save comparison as CSV
comparison_df.to_csv(f'../data/model_comparison.csv')

print(f"Models saved to {models_dir}")
print(f"Comparison saved to ../data/model_comparison.csv")

## 7. Feature Importance (for tree-based models)

In [ ]:
# Feature importance from Random Forest
rf_model = results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'Feature': pollutants,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../data/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFeature Importance:")
print(feature_importance)

## Summary

This notebook demonstrated:
1. **Data Generation**: Created synthetic AQI data with realistic pollutant distributions
2. **EDA**: Analyzed distributions, correlations, and category breakdowns
3. **Preprocessing**: Scaled features and encoded labels
4. **Model Training**: Trained 5 different classifiers
5. **Evaluation**: Compared models using accuracy, precision, recall, F1, and cross-validation
6. **Model Selection**: Automatically selected the best model based on F1 score
7. **Saving**: Saved all models and the scaler for production use

The best performing model has been saved and can be loaded by the API for real-time predictions.